# MatchaEdit Helper
### 2026 Made with ❤️, Gilang Wahyu Prasetyo
Mo bulk edit GC? pake Tools ini ges

# 1. Improt

In [ ]:
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
pd.set_option("display.max_columns", None)



# 2. Buka Chrome

In [ ]:
url = "https://matchapro.web.bps.go.id/dirgc"

chrome_cust = "isi di sini path nya"

user_agent = (
    "Mozilla/5.0 (Linux; Android 15; SM-G991B Build/TP1A.220624.014; wv) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 "
    "Chrome/132.0.6834.88 Mobile Safari/537.36"
)

options = Options()
options.add_argument("--start-maximized")
options.add_argument(f"--user-agent={user_agent}")

try:
    service = Service(executable_path=chrome_cust)
    driver = webdriver.Chrome(service=service, options=options)
except Exception:
    service = Service()
    driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

# 3. Login SSO
masukin OTP dulu

In [ ]:
username_sso = "sso"
password_sso = "pass"

wait = WebDriverWait(driver, 20)

# Klik tombol SSO
wait.until(EC.element_to_be_clickable((By.ID, "login-sso"))).click()

# Isi kredensial
wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username_sso)
driver.find_element(By.ID, "password").send_keys(password_sso)

# Submit login
driver.find_element(By.ID, "kc-login").click()

# 4. Ambil Cookie

In [ ]:
# Tunggu login selesai
time.sleep(5)

# Ambil cookies dari Selenium
cookies_selenium = driver.get_cookies()

# Ambil CSRF token
csrf_token = driver.execute_script("""
    return document.querySelector('meta[name="csrf-token"]').content;
""")

print(f"CSRF Token: {csrf_token}")

# Convert cookies ke requests session
session = requests.Session()
for cookie in cookies_selenium:
    session.cookies.set(cookie['name'], cookie['value'])


# 5. Import Hasil Matchaget

In [ ]:
url = 'masukkan link spreadsheet mu yang langsung dari url'

sheet_id = url.split('/d/')[1].split('/')[0]
gid = url.split('gid=')[1].split('#')[0]

hasilgc = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}')
hasilgc = hasilgc[
    (hasilgc["gc_username"] == "gilangprasetyo")
    # & (hasilgc["gcs_result"] == 1) &
    # & (hasilgc["latlong_status_gc"] == "invalid")
].reset_index(drop=True)

print(f"✅ Imported & filtered {len(hasilgc)} records")
print(hasilgc[['perusahaan_id','latitude_gc','longitude_gc','idsbr']].head())

# 6. Fix Coordinate
ini cuma buat yang titiknya banyak, atau pemisahnya masih pake koma. untuk yang string/nan/kosong, bisa ubah sendiri manual atau pake cekSBR Helper

In [ ]:
def fix_coord(val):
    if pd.isna(val):
        return np.nan
    
    val = str(val).replace(",", ".")
    
    # ambil hanya minus dan angka
    digits = re.sub(r"[^0-9\-]", "", val)
    
    if digits in ["", "-"]:
        return np.nan
    
    sign = "-" if digits.startswith("-") else ""
    digits = digits.lstrip("-")
    
    if len(digits) <= 3:
        return np.nan
    
    fixed = sign + digits[:3] + "." + digits[3:]
    
    try:
        return float(fixed)
    except:
        return np.nan

def fix_lat(val):
    if pd.isna(val):
        return np.nan
    
    digits = re.sub(r"[^0-9\-]", "", str(val))
    if digits in ["", "-"]:
        return np.nan
    
    if digits.startswith("-"):
        sign = "-"
        digits = digits[1:]
        if len(digits) < 2:
            return np.nan
        fixed = sign + digits[:1] + "." + digits[1:]
    else:
        if len(digits) < 1:
            return np.nan
        fixed = digits[:1] + "." + digits[1:]
    
    try:
        return float(fixed)
    except:
        return np.nan

hasilgc["latitude_gc"] = hasilgc["latitude_gc"].apply(fix_lat)
hasilgc["longitude_gc"] = hasilgc["longitude_gc"].apply(fix_coord)


# 7. Bulk Edit

## def 

In [ ]:
def editsbr(df, driver):
    """
    Bulk update SBR via Selenium - REPEAT rate limit row setelah wait 30s.
    """
    df = df.reset_index(drop=True)
    print(f"Total records sekarang: {len(df)}")
    
    csrf_token = driver.execute_script("return document.querySelector('meta[\\\"csrf-token\\\"]').content")
    success_count = failed_count = rate_limit_count = 0
    total = len(df)
    
    print(f"🚀 Start bulk update {total} records...")
    
    idx = 0
    while idx < total:
        row = df.iloc[idx]
        perusahaan_id = row['perusahaan_id']
        idsbr = row.get('idsbr', 'N/A')
        lat_gc = str(row['latitude_gc'])
        lng_gc = str(row['longitude_gc'])
        hsl_gc = int(row['gcs_result'])
        
        print(f"[{idx+1}/{total}] ID: {perusahaan_id[:20]}... | SBR: {idsbr}")
        
        gc_token = driver.execute_script("return document.body.innerHTML.match(/new_gc_token[^\\\"]*\\\"([^\\\"]*)\\\"/i)?.[1] || ''")
        
        result = driver.execute_script(f"""
            return fetch('https://matchapro.web.bps.go.id/dirgc/konfirmasi-user', {{
                method: 'POST',
                headers: {{'X-Requested-With': 'XMLHttpRequest'}},
                body: new URLSearchParams({{
                    '_token': '{csrf_token}',
                    'perusahaan_id': '{perusahaan_id}',
                    'latitude': '{lat_gc}',
                    'longitude': '{lng_gc}',
                    'hasilgc': '{hsl_gc}',
                    'gc_token': '{gc_token}',
                    'edit_nama': '0',
                    'edit_alamat': '0',
                    'nama_usaha': '',
                    'alamat_usaha': '',
                    'time_on_page': '0'
                }})
            }}).then(r => r.json());
        """)
        
        if result.get('status') == 'success' or 'Berhasil' in result.get('message', ''):
            success_count += 1
            print(f"✅ [{idx+1}] OK")
            idx += 1  # Next row
            
        elif 'Too Many Attempts' in str(result):
            rate_limit_count += 1
            print(f"⏸️ [{idx+1}] Rate limit - wait 30s & RETRY")
            time.sleep(30)
            # idx tidak +1, ulangi baris ini
            
        else:
            print(f"❌ [{idx+1}] {result.get('message', 'Error')}")
            failed_count += 1
            idx += 1  # Next row
    
    print(f"\n🎉 SUCCESS: {success_count} | FAILED: {failed_count} | RATE LIMIT RETRY: {rate_limit_count}")
    return success_count, failed_count, rate_limit_count


## filter hasilgc

In [ ]:
fhasilgc = pd.DataFrame()  # Empty DF
# sekali dirun aja

In [ ]:
# 1. Pencarian case insensitive exact match di kolom 'nama_usaha'
usaha = ""  # Keyword nama_usaha
alamat = "food court"  # Keyword alamat_usaha (ubah di sini)

shasilgc = hasilgc[
    hasilgc['nama_usaha'].str.contains(usaha, case=False, na=False, regex=False)
    & hasilgc['alamat_usaha'].str.contains(alamat, case=False, na=False, regex=False)
].reset_index(drop=True)

print(f"Hasil pencarian '{usaha}' di '{alamat}': {len(shasilgc)} records")
shasilgc

In [ ]:
n = 2  # Ubah jumlah baris di sini
# shasilgc = shasilgc.iloc[n:].reset_index(drop=True) # atas
# shasilgc = shasilgc.iloc[:-n].reset_index(drop=True) # bawah
shasilgc = shasilgc.drop(shasilgc.index[n-1]).reset_index(drop=True) # ke-n dari atas
shasilgc

In [ ]:
shasilgc['gcs_result'] = 99

In [ ]:
# 2. Append shasilgc
fhasilgc = pd.concat([fhasilgc, shasilgc], ignore_index=True)

print(f"fhasilgc sekarang: {len(fhasilgc)} records")


## eksekusi

In [ ]:
# Usage
editsbr(fhasilgc, driver)